In [4]:
# Cell 1: 准备环境与数据
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import os

# 1. 锁定 M4 的 MPS 加速
device = torch.device("mps")
print(f"🔥 硬件加速已锁定: {device} (Apple M4)")

# 2. 读取数据
filename = 'processed_data.csv'
if os.path.exists(os.path.join('..', 'data', filename)):
    df = pd.read_csv(os.path.join('..', 'data', filename))
else:
    df = pd.read_csv(os.path.join('data', filename))

# 3. 标签处理
df = df.dropna(subset=['cleaned_text', 'labels'])
label_map = {'bad': 0, 'neutral': 1, 'good': 2}
df['label_id'] = df['labels'].map(label_map)

# 4. 加载 Tokenizer (DistilBERT)
# 第一次运行会自动下载字典，约 200MB
print("⏳ 正在加载 DistilBERT Tokenizer...")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')#不区分大小写uncased

# 5. 定义 Dataset 类 (核心)
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # 核心：使用 BERT 的 tokenizer 编码
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 6. 划分数据
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label_id'], test_size=0.2, random_state=42)

# 创建 DataLoader
# batch_size=32 是 M4 16GB 内存的安全值，如果报错内存不足，改 16
train_dataset = SentimentDataset(X_train.to_numpy(), y_train.to_numpy(), tokenizer)
test_dataset = SentimentDataset(X_test.to_numpy(), y_test.to_numpy(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✅ 数据装载完毕！训练集: {len(train_dataset)}, 测试集: {len(test_dataset)}")

🔥 硬件加速已锁定: mps (Apple M4)
⏳ 正在加载 DistilBERT Tokenizer...
✅ 数据装载完毕！训练集: 175428, 测试集: 43858


In [5]:
from transformers import DistilBertForSequenceClassification

model_path = "./model_save"  # 你之前保存模型的目录
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.to(device)

print("✅ 已从 model_save 加载微调后的 DistilBERT 模型")


✅ 已从 model_save 加载微调后的 DistilBERT 模型


In [5]:
# Cell 2 (修正版): 训练 DistilBERT (适配 Mac MPS float32)

# 加载预训练模型
print("⏳ 正在加载预训练模型权重 (DistilBERT)...")
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', 
    num_labels=3
)
model = model.to(device) # 搬到 GPU

# 优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# === 训练开始 ===
EPOCHS = 1 
print(f"🚀 开始训练！计划轮数: {EPOCHS}")
print("💡 提示：你可以打开 Activity Monitor 观察 GPU 占用率")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct_predictions = 0
    n_examples = 0
    
    for i, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 统计
        _, preds = torch.max(logits, dim=1)
        total_loss += loss.item()
        
        # ⚠️⚠️⚠️ 修正点：使用 .float() 代替 .double()
        correct_predictions += torch.sum(preds == labels)
        n_examples += labels.size(0)
        
        if i % 100 == 0 and i > 0:
            # 这里的计算逻辑也改为了 float
            curr_acc = correct_predictions.float() / n_examples
            print(f"Epoch {epoch+1} | Batch {i}/{len(train_loader)} | Loss: {loss.item():.4f} | Acc: {curr_acc.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    # ⚠️⚠️⚠️ 修正点：使用 .float()
    avg_acc = correct_predictions.float() / n_examples
    print(f"🏆 Epoch {epoch+1} 完成! Average Loss: {avg_loss:.4f} | Average Acc: {avg_acc.item():.4f}")

⏳ 正在加载预训练模型权重 (DistilBERT)...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 开始训练！计划轮数: 1
💡 提示：你可以打开 Activity Monitor 观察 GPU 占用率
Epoch 1 | Batch 100/5483 | Loss: 0.6954 | Acc: 0.5724
Epoch 1 | Batch 200/5483 | Loss: 0.5719 | Acc: 0.6502
Epoch 1 | Batch 300/5483 | Loss: 0.5356 | Acc: 0.6894
Epoch 1 | Batch 400/5483 | Loss: 0.6521 | Acc: 0.7126
Epoch 1 | Batch 500/5483 | Loss: 0.6119 | Acc: 0.7297
Epoch 1 | Batch 600/5483 | Loss: 0.4354 | Acc: 0.7434
Epoch 1 | Batch 700/5483 | Loss: 0.5078 | Acc: 0.7554
Epoch 1 | Batch 800/5483 | Loss: 0.4188 | Acc: 0.7654
Epoch 1 | Batch 900/5483 | Loss: 0.6083 | Acc: 0.7742
Epoch 1 | Batch 1000/5483 | Loss: 0.5006 | Acc: 0.7807
Epoch 1 | Batch 1100/5483 | Loss: 0.4335 | Acc: 0.7874
Epoch 1 | Batch 1200/5483 | Loss: 0.2063 | Acc: 0.7946
Epoch 1 | Batch 1300/5483 | Loss: 0.4786 | Acc: 0.8005
Epoch 1 | Batch 1400/5483 | Loss: 0.2565 | Acc: 0.8061
Epoch 1 | Batch 1500/5483 | Loss: 0.3125 | Acc: 0.8101
Epoch 1 | Batch 1600/5483 | Loss: 0.3931 | Acc: 0.8142
Epoch 1 | Batch 1700/5483 | Loss: 0.1843 | Acc: 0.8180
Epoch 1 | Batch 1800

In [ ]:
# Cell 3: 最终评估
print("⏳ 正在评估测试集...")
model.eval()
predictions = []
real_values = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        
        predictions.extend(preds.cpu().numpy())
        real_values.extend(labels.cpu().numpy())

target_names = ['bad', 'neutral', 'good']
print("\n🤖 DistilBERT 最终战报:")
print(classification_report(real_values, predictions, target_names=target_names))

⏳ 正在评估测试集...

🤖 DistilBERT 最终战报:
              precision    recall  f1-score   support

         bad       0.97      0.95      0.96     21504
     neutral       0.86      0.89      0.87     11075
        good       0.93      0.93      0.93     11279

    accuracy                           0.93     43858
   macro avg       0.92      0.92      0.92     43858
weighted avg       0.93      0.93      0.93     43858



In [7]:
# Cell 4: 保存模型与 Tokenizer
import os

# 创建保存目录
output_dir = './model_save/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"💾 正在保存模型到 {output_dir} ...")

# 关键：我们要保存模型本身和对应的分词器
model_to_save = model.module if hasattr(model, 'module') else model  # 处理多卡情况(虽在你这也是单卡)
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ 模型已保存！文件夹里应该有 'config.json', 'pytorch_model.bin' 等文件。")
print("👉 队友如果要用，直接用 DistilBertForSequenceClassification.from_pretrained('./model_save/') 即可加载。")

💾 正在保存模型到 ./model_save/ ...
✅ 模型已保存！文件夹里应该有 'config.json', 'pytorch_model.bin' 等文件。
👉 队友如果要用，直接用 DistilBertForSequenceClassification.from_pretrained('./model_save/') 即可加载。


In [6]:
import torch.nn.functional as F
import pandas as pd

model.eval()
all_labels = []
all_preds = []
all_confidences = []

with torch.no_grad():
    for batch in test_loader:  # ✅ 用 test_loader 做评估
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        conf, preds = torch.max(probs, dim=-1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_confidences.extend(conf.cpu().numpy())

df_eval = pd.DataFrame({
    "label": all_labels,
    "pred": all_preds,
    "confidence": all_confidences
})

df_eval.to_csv("bert_eval_predictions.csv", index=False)
print("✅ 预测结果已保存到 bert_eval_predictions.csv")


✅ 预测结果已保存到 bert_eval_predictions.csv
